In [1]:
# 匯入套件和金鑰
import os
from rich import print as pprint
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from dotenv import load_dotenv
load_dotenv()

from langchain_community.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# LLM Model

In [12]:
# load LLM model
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY")) # "gpt-3.5-turbo", "gpt-4o"
embeddings_model=OpenAIEmbeddings(model='text-embedding-3-large')

# To Chroma

In [3]:
# load PDF file
pdf_filepath = '/Users/weikai/Library/CloudStorage/Dropbox/paper/CCWu'
pdf_filenames = [\
    'CCWu 2021 Modified distributed Bragg reflector for protecting organic light-emitting diode displays against ultraviolet light.pdf',
    'CCWu 2022 Analyses of emission efficiencies of white organic light-emitting diodes having multiple emitters in single emitting layer.pdf',
    'CCWu 2019 Three‐dimensional pixel configurations for optical outcoupling of OLED displays-optical simulation.pdf',
    'CCWu 2013 Analyzing nanostructures in mesogenic host–guest systems for polarized phosphorescence.pdf',
    'CCWu 2022 Analyses of emission efficiencies of white organic light-emitting diodes having multiple emitters in single emitting layer.pdf',
    'CCWu 2023 Fully electromagnetic wave optic simulation and analyses of the cross-scale reflective 3D OLED pixel configuration.pdf',
    'CCWu 2022 Using angle-selective optical film to enhance the light extraction of a thin-film encapsulated 3D reflective pixel for OLED displays.pdf',
    'CCWu 2022 P-128 Optimizing OLED Pixel Structures for Consistently Low Ambient Light Reflection over Viewing Angles.pdf',
    'CCWu 2020 Integrating Molecular Rigidity and Chirality into TADF for Highly Efficient Sky-Blue CPEL.pdf',
    'CCWu 2020 High-efficiency organic light emitting diodes using high-index transparent electrode.pdf',
    'CCWu 2019 SID P-179 Optics of Curved OLEDs.pdf',
    'CCWu 2020 Enhance external quantum efficiency of organic light-emitting devices using thin transparent electrodes.pdf',
    'CCWu 2016 Triboluminescence and Metal Phosphor for Organic Light-Emitting Diodes Functional Pt(II) Complexes with Both 2‑Pyridylimidazol-2-ylidene and Bipyrazolate Chelates.pdf',
    'CCWu 2018 Quantitative analyses of high electroluminescence efficiency of thermally activated delayed fluorescence emitters based on acridine–triazine hybrids.pdf',
    'CCWu 2017 Achieving Nearly 30% External Quantum Efﬁciency for Orange–Red Organic Light Emitting Diodes by Employing Thermally Activated Delayed Fluorescence Emitters  Composed of 1,8-Naphthalimide-Acridine Hybrids.pdf',
    'CCWu 2016 Efﬁcient and Tunable Thermally Activated Delayed Fluorescence Emitters Having Orientation-Adjustable  CN-Substituted Pyridine and Pyrimidine Acceptor Units.pdf'
]

# pdf_filepath = '/Users/weikai/Library/CloudStorage/Dropbox/paper/CCWu'
# pdf_filenames = [ fname for fname in os.listdir(pdf_filepath) if fname.endswith('.pdf') ]


In [4]:
docs = []
for pdf_filename in tqdm(pdf_filenames):
    loader = PyPDFLoader(file_path=os.path.join(pdf_filepath, pdf_filename))
    docs += loader.load()
print(f"Loaded {len(docs)} documents")

100%|██████████| 16/16 [00:11<00:00,  1.41it/s]

Loaded 182 documents


In [5]:
database_path = './database/vector_db_chroma/'
# Chroma.from_documents(documents=docs,
#                       embedding=embeddings_model,
#                       persist_directory=database_path,
#                       collection_metadata={"hnsw:space": "cosine"})

# load Chroma
db = Chroma(persist_directory=database_path, 
            embedding_function=embeddings_model)

# Retrieve

In [6]:
question = "how to protect OLED display against UV light?"
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(question)
print(f'傳回 {len(retrieved_docs)} 筆資料')
# pprint( retrieved_docs )

傳回 5 筆資料


# Question-Answer

In [13]:
str_parser = StrOutputParser()
template = (
    "請根據以下內容加上自身判斷回答問題 (請用繁體中文回答/專有名詞則使用英文):\n"
    "{context}\n"
    "問題: {question}"
    )
prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | str_parser
)

In [14]:
questions = [\
    "how to protect OLED display against UV light?", 
    "What is the key idea of 3D pixel OLED?",
    "How to achieve high emission efficient OLED?",
    "How to fabricate high emission OLED pixel device? What is the possible structure and material?",
    "What is the issue to fabricate high efficient red OLED?",
    "What is the optical issue in curve OLED?",
    "Can you summarize the outcoupling techniques for OLED?",
]

for ii, question in enumerate( questions ):
    pprint(f'問題-{ii+1}: {question}')
    pprint(f'回答-{ii+1}: {chain.invoke(question)}')
    pprint('')

問題-1: how to protect OLED display against UV light?

回答-1: 要保護OLED顯示屏免受紫外線（UV）光的影響，可以採取以下措施：

1. **使用抗UV/HEV結構**：在OLED顯示屏的保護結構中引入抗UV/HEV的功能層。根據文獻，將這些結構整合到保護玻璃（cover 
lens）中是最理想的選擇。由於保護玻璃位於OLED顯示屏的最外層，不會直接影響OLED的發光性能。

2. 
**改良布拉格反射器（DBR）結構**：文獻中提到，使用改良過的布拉格反射器（DBR）結構，其中包含UV吸收的介電材料，並調整
層/對的厚度，以實現高效的UV阻擋效果（在400nm以下的透射率接近0%），並在可見光範圍內保持高透射率（約92%）。這種結構可
以有效防止UV光對OLED顯示屏造成的降解。

3. 
**使用圓偏振片**：圓偏振片可以部分阻擋UV/HEV光，但需注意的是長期暴露在UV/HEV光下會使圓偏振片本身降解，因此這種方法
更適合於室內或短期的戶外應用。

4. 
**結構設計的策略**：在設計抗UV/HEV結構時，應考慮在410-420nm以下的UV/HEV阻擋區域內，透射率應保持在低（理想情況下為零
）；同時，在440-450nm以上的可見光範圍內，透射率應保持高，以確保不影響OLED顯示屏的R/G/B顏色性能。

5. 
**實驗驗證**：根據文獻，實驗測試證實這些抗UV結構可以有效減少UV/太陽光引起的降解現象，從而保護OLED顯示屏的性能，例如
減少電壓漂移等。

總結來說，保護OLED顯示屏免受UV光影響的最佳方法是在顯示屏的保護玻璃上集成抗UV/HEV結構，這些結構應使用更穩定的無機材
料，並且需經過實驗驗證其有效性。

問題-2: What is the key idea of 3D pixel OLED?

回答-2: 3D pixel 
OLED的關鍵概念是通過引入三維像素結構來增強光的出耦合效率。傳統AMOLED像素結構中的主要光學損失來自於波導模式（WG模式
）和表面等離子體模式（SP模式），這使得其外量子效率（EQE）通常只有約20%。為了解決這個問題，研究者提出了一種三維像素
配置，具體包括以下幾個關鍵點：

1. 
**底部反射電極延伸**：將底部反射電極延伸到像素定義層（PDL）的斜坡上，形成一個凹形結構，這樣可以更有效地引導內部生成
的光子進入高折射率填充區域。

2. **高折射率填充材料**：在凹形區域填充高折射率材料（折射率n ≥ 
1.8），通過選擇性沉積方法（如噴墨打印、蒸汽噴射打印、精細掩模蒸鍍等）進行圖案化填充。這些材料有助於將內部生成的光子
耦合進入高折射率填充區域。

3. 
**光的多重反射和重定向**：利用凹形結構的反射特性，將原本被限制的光線（即進入填充材料的內部角度θ_int大於填充材料-空
氣界面的全內反射臨界角θ_c的光線）通過一個或多個反射重定向到外部，從而提高光的出耦合效率。

4. **波長不敏感的提取機制**：這種光提取機制對波長不敏感，因此對所有R/G/B顏色均有效。

5. 
**適合高解析度顯示**：隨著像素/子像素尺寸縮小，光線在厚的高折射率填充層中的反射次數減少，從而減少了底部/頂部電極和
其他吸收層的吸收損失，進而在更小的子像素/更高解析度下達到更高的光出耦合效率。

總的來說，3D像素OLED通過結構設計和材料選擇，顯著改進了光的出耦合效率，從而提升了AMOLED顯示的性能和能效。

問題-3: How to achieve high emission efficient OLED?

...

回答-5: 製造高效紅色有機發光二極體（OLED）面臨的主要問題包括：

1. 
**分子設計複雜性**：紅色OLED需要能夠有效上轉換從三重態（T1）到單重態（S1）的發射材料，這需要精確設計具有小能隙（ΔES
T）和適當前沿分子軌道重疊的分子。

2. 
**光致發光量子產率（ΦPL）**：紅色OLED的發射材料必須具有高光致發光量子產率，以確保高效的光輸出。然而，設計出同時具有
高ΦPL和合適能級結構的分子非常困難。

3. 
**水平發射偶極取向**：為了提高OLED的外部量子效率（EQE），發射材料的偶極子取向最好是水平的。然而，實現這種取向在分子
設計和材料製造上都是一個挑戰。

4. **微腔效應**：紅色OLED的效率還受到微腔效應的影響，需要在設計和製造過程中考慮這些效應以優化器件性能。

5. 
**少量報告的高效紅色TADF發射體**：目前報導的高效紅色TADF（熱活化延遲螢光）發射體很少，這限制了紅色OLED的進一步發展
。紅色TADF OLED的發展遠遠落後於藍色和綠色TADF OLED。

總的來說，製造高效紅色OLED需要克服在分子設計、材料選擇和製造工藝上的多重挑戰。

問題-6: What is the optical issue in curve OLED?

回答-6: 根據提供的文件內容，曲面OLED的光學問題主要有以下幾個方面：

1. **光提取效率（Light Extraction Efficiency）**：
   - 
曲面OLED的光提取效率會受到曲率的影響。例如，球面凸曲面（spherical-convex）結構能顯著提高光提取效率，達到接近100%。
然而，球面凹曲面（spherical-concave）結構可能會導致光提取效率降低。

2. **發光角度和出射角度的變化（Emission and Exit Angles）**：
   - 
在曲面OLED中，發光角度與出射角度之間存在差異，這會影響光的出射效率和角度分布。例如，凸曲面結構會擴大有效的臨界角，
可能導致光提取效率顯著提高，而凹曲面結構可能會減少光提取。

3. **影像幾何變形（Geometric Distortion）**：
   - 
曲面結構會導致影像的幾何變形，這會影響顯示品質和觀看體驗。例如，曲面顯示器上的影像邊緣可能會被壓縮或拉伸，導致觀察
到的影像變形。

4. **亮度減少（Brightness Reduction）**：
   - 曲面結構會導致影像邊緣亮度降低，這是由於不同發光角度和出射角度的影響。需要進行亮度校正以改善觀看體驗。

這些光學問題對於曲面OLED顯示器的設計和應用至關重要，需要進行仔細研究和考量，以提升顯示品質和使用者體驗。

問題-7: Can you summarize the outcoupling techniques for OLED?

回答-7: 有機發光二極體（OLED）的光耦合技術是提高其光提取效率（Light Extraction Efficiency, 
LEE）的關鍵。以下是幾種常見的光耦合技術：

1. **高指數填充區域**：
   - 使用高指數填充材料來替代空氣，提升內部光的耦合效率。這種方法通過計算OLED內部輻射耦合到高指數填充區域的效率（η 
filler）來實現。

2. **光子晶體**：
   - 
在OLED表面引入光子晶體結構，如SiO2/SiNx光子晶體，可以增強光提取。光子晶體能有效地改變光的傳播方向，減少內部全反射的
損失。

3. **納米結構電極**：
   - 使用納米結構的複合電極，如ITO納米網（Indium Tin Oxide 
Nanomesh）和導電聚合物的結合，提高光的耦合效率。這些納米結構可以減少電極對光的阻擋和吸收。

4. **表面等離子體共振（Surface Plasmon Polaritons, SPP）**：
   - 利用SPP模式進行能量轉移，增強光的提取效應。這種技術通過在有機層和金屬層之間引入SPP效應來實現。

5. **多層結構和微腔效應**：
   - 優化OLED的多層結構，如調整空穴傳輸層（HTL）和電子傳輸層（ETL）的厚度，並利用微腔效應（Microcavity 
Effect），使發光層（EML）位於反射金屬電極的第二反節點處，提高光的耦合效率。

6. **低折射率層**：
   - 在OLED表面添加低折射率層，通過納米壓印技術（Nanoimprinting Technique）創建低折射率結構，增強正向光的耦合。

7. **3D像素配置**：
   - 
採用三維（3D）像素結構，通過綜合考慮角度、光譜和偏振依賴性的3D光線追踪模擬，來提高光從高指數填充區域到空氣的提取效
率（η air）。

這些技術的綜合應用，可以顯著提高OLED的光提取效率，從而提高其整體亮度和能效。